# [모듈 3.1] Code Pipeline 생성 하기

## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정 


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

## 1.0 기존의 변수 불러오기

In [ ]:
%store -r code_pipeline_serving_config_json_path
%store -r sm_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))



# 2. 변수 정의

In [ ]:
region = code_pipeline_serving_dict["region"]
repository_name = code_pipeline_serving_dict["serving_code_repo_name"]
code_pipeline_name = code_pipeline_serving_dict["code_pipeline_name"]
code_build_project_name = code_pipeline_serving_dict["code_build_project_name"]
branch_name = code_pipeline_serving_dict["branch_name"]
pipeline_role_name = code_pipeline_serving_dict["code_pipeline_role_arn"]
bucket = code_pipeline_serving_dict["bucket"]


# 4. 파이프라인 JSON 정의

In [ ]:
pipeline_dic = {
    "pipeline": {
        "name": code_pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 4.1. Dic 포맷을 JSON 으로 변경

In [ ]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

## 4.2. JSON 파일 저장

In [ ]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [ ]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

# 5. 파이프라인 실행

In [ ]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

# 6. 코드 파이프라인 실행 상태 확인

In [ ]:
%%sh -s {code_pipeline_name}
code_pipeline_name=$1
echo $code_pipeline_name
aws codepipeline get-pipeline-state --name $code_pipeline_name

# 7. 실행 확인

## 7.1. Code Pipeline 단계 확인

![serving_codepipeline_stage.png](img/serving_codepipeline_stage.png)

## 7.2. SageMaker Serving Pipeline

![ncf_serving_sm_pipeline.png](img/ncf_serving_sm_pipeline.png)

## 7.3. SageMaker Pipeline Step 확인

![ncf_serving_step.png](img/ncf_serving_step.png)

## 7.4. NCF 엔드포인트 확인

![ncf_endpoint.png](img/ncf_endpoint.png)